In [1]:
import sys
import os

# Agregar el directorio raíz del proyecto a sys.path
project_root = "/home/javitrucas/TFG"
if project_root not in sys.path:
    sys.path.append(project_root)

import torch
import wandb
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.dataset_loader import load_dataset
from scripts.medical_scripts.medical_evaluation import ModelEvaluator
from scripts.medical_scripts.medical_training import Training
from scripts.MIL_utils import MIL_collate_fn

from types import SimpleNamespace
from box import Box

/home/javitrucas/miniconda3/envs/tfg/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/javitrucas/miniconda3/envs/tfg/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def run_experiment(config):
    # Crear un objeto temporal para compatibilidad
    class TempConfig:
        def __init__(self, config_dict):
            self.__dict__.update(config_dict)

    config_obj = TempConfig(config)

    # Iniciar wandb
    wandb.init(
        project="TFG",
        config=config
    )

    # Parámetros controlados desde la configuración
    dataset_name = config_obj.dataset_name
    num_epochs = config_obj.num_epochs
    learning_rate = config_obj.learning_rate
    batch_size = config_obj.batch_size
    pooling_type = config_obj.pooling_type       
    input_feature_dim = config_obj.input_feature_dim
    feature_dim = config_obj.feature_dim

    # Directorio para guardar modelos
    output_model_dir = f"./models/{dataset_name.split('-')[0]}"
    os.makedirs(output_model_dir, exist_ok=True)

    # Cargar datasets
    train_dataset, val_dataset = load_dataset(config=config, mode="train_val")
    test_dataset = load_dataset(config=config, mode="test")

    # Crear dataloaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=MIL_collate_fn)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=MIL_collate_fn)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=MIL_collate_fn)

    # Iniciar el entrenamiento
    trainer = Training(
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=num_epochs,
        learning_rate=learning_rate,
        output_model_dir=output_model_dir,
        pooling_type=pooling_type,              # Pasar pooling_type
        input_feature_dim=input_feature_dim,    # Pasar input_feature_dim
        feature_dim=feature_dim,                # Pasar feature_dim
        wandb=wandb
    )
    trainer.train()

    # Guardar el modelo correctamente antes de evaluarlo
    model_path = os.path.join(output_model_dir, 'model.pth')
    torch.save(trainer.model.state_dict(), model_path)  # Asegurar que el modelo se guarda

    # Verificar que el archivo se haya guardado correctamente
    if not os.path.isfile(model_path):
        raise ValueError(f"El modelo no se guardó correctamente en: {model_path}")

    # Guardar el modelo como un artefacto en wandb
    artifact = wandb.Artifact('model', type='model')
    artifact.add_file(model_path)
    wandb.log_artifact(artifact)

    # Iniciar la evaluación
    evaluator = ModelEvaluator(
        model_path=model_path,
        test_loader=test_loader,
        batch_size=batch_size,
        input_feature_dim=input_feature_dim,  
        feature_dim=feature_dim,              
        pooling_type=pooling_type,
        wandb=wandb
    )
    # Iniciar la evaluación
    metrics, attention_weights_list = evaluator.evaluate()  # Se desempaquetan correctamente los valores

    # Métricas incluidas
    results = {
        "train_loss_curve": trainer.train_loss_curve,
        "val_loss_curve": trainer.val_loss_curve,
        "train_accuracy_curve": trainer.train_accuracy_curve,
        "val_accuracy_curve": trainer.val_accuracy_curve,
        "test_loss_curve": evaluator.test_loss_curve,
        "test_accuracy_curve": evaluator.test_accuracy_curve,
        "train_f1_score": trainer.train_f1_score,
        "val_f1_score": trainer.val_f1_score,
        "test_f1_score": evaluator.test_f1_score,
        "train_auc_roc": trainer.train_auc_roc,
        "val_auc_roc": trainer.val_auc_roc,
        "test_auc_roc": evaluator.test_auc_roc,
        "train_precision": trainer.train_precision,
        "val_precision": trainer.val_precision,
        "test_precision": evaluator.test_precision,
        "train_recall": trainer.train_recall,
        "val_recall": trainer.val_recall,
        "test_recall": evaluator.test_recall,
        "confusion_matrix": metrics["confusion_matrix"]  # Se accede correctamente a metrics
    }

    # Finalizar wandb
    wandb.finish()


In [3]:
# Convertir config_1 en un objeto usando Box
config_1 = Box({
    "dataset_name": "rsna-features_resnet18",
    "input_feature_dim": 512,  # Características de ResNet50
    "feature_dim": 128,        # Dimensión final después de reducción
    "pooling_type": "attention",
    "num_epochs": 50,
    "learning_rate": 1e-3,
    "batch_size": 1,
    "val_prop": 0.2,
    "seed": 42,
    "use_inst_distances": False,
    "adj_mat_mode": "relative"
})

# Ejecutar el experimentoTFG
run_experiment(config_1)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: javitrucas (javitrucas-universidad-de-granada) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


rsna
[RSNADataset] Scanning files...
[RSNADataset] Found 1000 already processed bags
[RSNADataset] Number of bags found: 1000
rsna
[RSNADataset] Scanning files...
[RSNADataset] Found 150 already processed bags
[RSNADataset] Number of bags found: 150
Epoch 1/50
Train - Loss: 0.7003, Acc: 0.6162, AUC: 0.6278, F1: 0.4770
Val   - Loss: 0.5861, Acc: 0.6950, AUC: 0.7532, F1: 0.5197
Final model saved to ./models/rsna/model_attention.pth
Epoch 2/50
Train - Loss: 0.6071, Acc: 0.6987, AUC: 0.7340, F1: 0.6043
Val   - Loss: 0.8087, Acc: 0.5000, AUC: 0.7801, F1: 0.6124
Final model saved to ./models/rsna/model_attention.pth
Epoch 3/50
Train - Loss: 0.6058, Acc: 0.6900, AUC: 0.7526, F1: 0.5948
Val   - Loss: 0.5291, Acc: 0.7200, AUC: 0.8012, F1: 0.6410
Final model saved to ./models/rsna/model_attention.pth
Epoch 4/50
Train - Loss: 0.5469, Acc: 0.7350, AUC: 0.7873, F1: 0.6570
Val   - Loss: 0.9158, Acc: 0.6350, AUC: 0.8065, F1: 0.1978
Final model saved to ./models/rsna/model_attention.pth
Epoch 5/50
Tra

/home/javitrucas/TFG/scripts/medical_scripts/medical_evaluation.py:64: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([''] + ["Negative", "Positive"])
/home/javitrucas/TFG/scripts/medical_scripts/medical_evaluation.py:65: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels([''] + ["Negative", "Positive"])


Confusion matrix saved at output/attention/confusion_matrix.png

--- Evaluation Results ---
Optimal Threshold: 0.3089
Accuracy: 0.8000, AUC: 0.8485
Precision: 0.8182, Recall: 0.7500, F1-Score: 0.7826
Confusion Matrix:
[[66 12]
 [18 54]]
Attention heatmap 0 saved at output/attention/attention_heatmap_0.png
Attention heatmap 1 saved at output/attention/attention_heatmap_1.png
Attention heatmap 2 saved at output/attention/attention_heatmap_2.png
Attention heatmap 3 saved at output/attention/attention_heatmap_3.png
Attention heatmap 4 saved at output/attention/attention_heatmap_4.png


epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
test_accuracy,▁
test_auc,▁
test_f1,▁
test_precision,▁
test_recall,▁
train_accuracy,▁▄▄▆▆▆▆▆▇▇█▆█████▇
train_auc,▁▄▅▆▆▆▇▆▆▇▇▇█▇████
train_f1,▁▄▄▆▆▆▆▆▇▇█▆▇█▇██▇
train_loss,█▅▅▄▃▃▃▄▃▂▂▂▁▂▁▁▁▂
val_accuracy,▆▁▇▄▃▅█▇███▇▇▆▇█▆█


In [ ]:
config_panda = Box({
    "dataset_name": "panda-patches_512_preset-features_UNI",
    "input_feature_dim": 1024,  # Dimensionalidad de las características pre-extraídas
    "feature_dim": 128,         # Dimensionalidad final después de reducción
    "pooling_type": "attention",  # Tipo de pooling (puede ser 'attention', 'mean', 'max')
    "num_epochs": 15,
    "learning_rate": 1e-3,
    "batch_size": 1,
    "val_prop": 0.2,
    "seed": 42,
    "use_inst_distances": False,
    "adj_mat_mode": "relative"
})

# Ejecutar el experimento
run_experiment(config_panda)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: javitrucas (javitrucas-universidad-de-granada) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


panda
[WSIDataset] Scanning files...


[WSIDataset] Building data dict:   2%|▏         | 207/8822 [01:07<52:03,  2.76it/s]  